In [ ]:
#set adequate flag for Theano on lxplus
import theano
theano.config.gcc.cxxflags = '-march=corei7'

In [ ]:
#load needed things
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.layers import Input, Activation, Dense, Convolution2D, MaxPooling2D, Dropout, Flatten
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pyp
import ROOT
import itertools
import math

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
#format the inputs from TTree
# 4D tensor (theano backend)
# 1st dim is jet index
# 2nd dim is eta bin
# 3rd dim is phi bin
# 4th dim is props value (pt, charge, pdgId, etc.)
def formatInputs(files):
    quark_jets = []#type 1
    gluon_jets = []#type 0

    for ifile in files:
        tree = ifile.Get('HZZ4LeptonsAnalysisReduced')
            
        for ievt, evt in enumerate(tree):
                    
    return signal, background

In [ ]:
#loads input data
vbf_inputs = ROOT.TFile.Open('/afs/cern.ch/work/m/mmelodea/private/MonoHiggs/CMSSW_9_0_0/src/JetImageFiles/VBF_HToZZTo4L_M125_13TeV_powheg2_JHUgenV6_pythia8.root')
ggh_inputs = ROOT.TFile.Open('/afs/cern.ch/work/m/mmelodea/private/MonoHiggs/CMSSW_9_0_0/src/JetImageFiles/GluGluHToZZTo4L_M125_13TeV_powheg2_JHUgenV6_pythia8.root')
inputs = [vbf_inputs,ggh_inputs]

#format tree inputs to adequate shape
quark_jets, gluon_jets = formatInputs(inputs)
print 'quark jets: %i' % len(quark_jets)
print 'gluon jets: %i' % len(gluon_jets)

In [ ]:
#filter events based on DR(eta,phi)
maxDr = 0.1 #jet radius - ak4PFJetCHS
fquark_jets = []
fgluon_jets = []

for ijet in quark_jets:
    if(ijet[0] < maxDr):
        fquark_jets.append([ijet[1],ijet[2],ijet[3],ijet[4],ijet[5]])
        
for ijet in gluon_jets:
    if(ijet[0] < maxDr):
        fgluon_jets.append([ijet[1],ijet[2],ijet[3],ijet[4],ijet[5]])
        
maxjets = min(len(fquark_jets),len(fgluon_jets))
fquark_jets = [fquark_jets[i] for i in range(maxjets)]
fgluon_jets = [fgluon_jets[i] for i in range(maxjets)]
print 'quark jets: %i (%.2f)' % (len(fquark_jets),len(fquark_jets)/float(len(quark_jets)))
print 'gluon jets: %i (%.2f)' % (len(fgluon_jets),len(fgluon_jets)/float(len(gluon_jets)))

In [ ]:
from keras.layers import LSTM, Bidirectional, Masking
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# Bidirectional LSTM Model
def build_bilstm_model(n_cand_per_jet, features_per_jet):
    # Headline input: meant to receive sequences of 200 floats
    # Note that we can name any layer by passing it a "name" argument.
    i = Input(shape=(n_cand_per_jet, features_per_jet,), name='main_input')
    # the masking layer will prevent the LSTM layer to consider the 0-padded jet values
    m = Masking()(i)

    # A LSTM will transform the vector sequence into a single vector,
    # containing information about the entire sequence
    # the Bidirectional will make the LSTM cell read the sequence from end to start and start to end at the same time
    m = Bidirectional( LSTM(100) ) (m)
    
    auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(m)
    model = Model(input=[i], output=[auxiliary_output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

## ADD THESE LINES TO PRINT MODEL SUMMARY
model = build_bilstm_model(200, 3)
print model.summary()

In [ ]:
njets = 1500
n_cand_per_jet = 200
features = {
    'ak4pfcand_pt' : None,
    'ak4pfcand_eta' : None,
    'ak4pfcand_phi' : None,
    'ak4pfcand_e' : None,
    'ak4pfcand_charge' : None
}
features_per_jet = len(features)

momentum_input = {}
momentum_input['quark'] = np.zeros((njets, n_cand_per_jet, features_per_jet)) # 3 momentum
momentum_input['gluon'] = np.zeros((njets, n_cand_per_jet, features_per_jet)) # 3 momentum

In [ ]:
for pfj in range(njets):
    for sbj in range(n_cand_per_jet):
        for prop in range(len(features)):
            if(sbj < len(fquark_jets[pfj][prop])):
                momentum_input['quark'][pfj][sbj][prop] = fquark_jets[pfj][prop][sbj]
            if(sbj < len(fgluon_jets[pfj][prop])):
                momentum_input['gluon'][pfj][sbj][prop] = fgluon_jets[pfj][prop][sbj]

#print momentum_input['VBF']
#print momentum_input['GGH']

In [ ]:
# Run classifier with cross-validation and plot ROC curves
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp

X = np.concatenate([momentum_input['quark'], momentum_input['gluon']])
Y_TT = np.ones(momentum_input['quark'].shape[0])
Y_QCD = np.zeros(momentum_input['gluon'].shape[0])
Y = np.concatenate([Y_TT, Y_QCD])

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

kfold = StratifiedKFold(n_splits=2, shuffle=True,  random_state=seed)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
colors = cycle(['cyan', 'indigo', 'seagreen', 'yellow', 'blue', 'darkorange', 'red', 'black', 'green', 'brown'])
lw = 2

i = 0
histories = []
for i,((train, test), color) in enumerate(zip(kfold.split(X, encoded_Y), colors)):
    print "\t\tFold",i
    bilstm_model = build_bilstm_model(n_cand_per_jet, features_per_jet)
    history = bilstm_model.fit(X[train], encoded_Y[train], 
                               validation_data=(X[test], encoded_Y[test]), 
                               nb_epoch=30, batch_size=128, 
                               verbose=1, callbacks=[early_stopping])
    Y_score = bilstm_model.predict(X[test])
    histories.append(history)
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(encoded_Y[test], Y_score)
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    pyp.plot(fpr, tpr, lw=lw, color=color, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

pyp.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k', label='Luck')
mean_tpr /= kfold.get_n_splits(X, encoded_Y)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
pyp.plot(mean_fpr, mean_tpr, color='g', linestyle='--',label='Mean ROC (area = %0.2f)' % mean_auc, lw=lw)
pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver operating characteristic example')
pyp.legend(loc="lower right")
pyp.show()